In [ ]:
import numpy as np
import os.path
import pandas
import matplotlib
import scipy.signal
import scipy.constants

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec

import mpld3
from mpld3 import plugins



## Load xviewer csv (from binary scope data) and scope csv

In [ ]:
def read_scope_xviewer_csv(filename):
    headerRows = 10        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:
                i1 = line.find(',')+1
                i2 = line.find(',',i1)                
                timeResolution = float( line[i1:i2].strip(' \t\n\r') )        
                numberOfCols = line.count(',')    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows,usecols=range(1,numberOfCols)).values
    return data, timeResolution


def read_scope_csv(filename):
    headerRows = 16        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:                
                timeResolution = float( line[line.rfind(',')+1:-1].strip(' \t\n\r') )    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows).values[:,1:-1]
    return data, timeResolution






In [ ]:
data_dir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170911/scope' 
file_name = "SCOPE_006.wdf.csv"  
filename = os.path.join(data_dir,file_name)

data, timeResolution = read_scope_xviewer_csv(filename)
time = timeResolution * np.arange(data.shape[0]) 

print '{} traces, time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
    data.shape[1], timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )

In [ ]:
# Raw data
plt.plot( data[0:5000,0] )
plt.title( 'photo diode signal (V)' )
plt.show()

plt.plot( data[0:5000,1] )
plt.title( 'voltage on laser fine-tuning (V)' )
plt.show()

plt.plot( data[0:5000,2] )
plt.title( 'montana sync signal (not used) (V)' )
plt.show()


In [ ]:
plt.plot( data[:,1], data[:,0], '.' )
plt.title( 'very naive way' )
plt.xlabel('transmission, photodiode  (V)')
plt.ylabel('laser fine-tuning (V)')
plt.show()

binDV = 0.5
binEdgesV = np.arange( -9, 9 + 0.5 * binDV, binDV )
binCentersV = binEdgesV[0:-2] + 0.5 * binDV
binN = len( binCentersV )
binMeanIntensityV = np.zeros( binN )

for i in range( binN ):
    ind = np.logical_and( data[:,1] >= binEdgesV[i], data[:,1] < binEdgesV[i+1] )    
    binMeanIntensityV[i] = np.mean( data[ind,0] )

plt.plot( binCentersV, binMeanIntensityV )
plt.title( 'slightly less naive way' )
plt.xlabel('laser fine-tuning in bins (V)')
plt.ylabel('mean transmission, photodiode  (V)')
plt.grid()
plt.show()

In [ ]:
# Better way: treat data of left and right flank (different movement of direction of laser piezo) separately.
#   1. Smooth laser fine-tuning data
#   2. Detect left and right flank of data. 
#   3. Plot the mean transmission as a function of binned laser fine-tuning voltages, separate left and right flanks of movement

windowWidth = 9 # should be ODD!
d = data[:,1]
data2 = data[ int(windowWidth-1)/2 : -int(windowWidth-1)/2, : ]
cumsumVec = np.cumsum(np.insert(d, 0, 0)) 
d = (cumsumVec[windowWidth:] - cumsumVec[:-windowWidth]) / windowWidth


In [ ]:
#   2. Detect left and right flanks. 
#      Find first value below Minlevel (we're now at the left side of a valley)
#      Move noiseLength points forward to make sure we're in the valley and not hampered by noise.
#      Find first value above Minlevel (we're now at the right side of a valley = left side of "left" flank )
#      Find first value above Maxlevel (we're now at the left side of a peak = right side of the "left" flank )
#      Move noiseLength points forward to make sure we're on the peak and not hampered by noise.
#      Etc..

dMinLevel = -7.5  # Minimal voltage of flank. For 9 V sweeping I used -7.5 V.  
dMaxLevel = 7.5   # Maximum voltage of flank. For 9 V sweeping I used -7.5 V. 
noiseLength = 20  # Number of time-steps that should be skipped after reaching min/max flank value.

maskLeft = np.zeros( d.shape[0], dtype=bool )
maskRight = np.zeros( d.shape[0], dtype=bool )

i = np.argmax( d < dMinLevel ) + noiseLength
while True:    
    i = np.argmax( d[i:] > dMinLevel ) + i     
    if i.size == 0:
        break
    iEnd = np.argmax( d[i:] > dMaxLevel ) + i
    if iEnd.size == 0: 
        break
    maskLeft[i:iEnd] = True    
    i = iEnd + noiseLength        
    if i >=  len( d ):
        break
    i = np.argmax( d[i:] < dMaxLevel ) + i    
    if i.size == 0:
        break
    iEnd = np.argmax( d[i:] < dMinLevel ) + i    
    if iEnd.size == 0:
        break
    maskRight[i:iEnd] = True
    i = iEnd + noiseLength    
    if i >=  len( d ):
        break

print 'left flank {}%, right flank {}%'.format( 100.0 * np.sum( maskLeft ) / len( maskLeft ), 100.0 * np.sum( maskRight ) / len( maskRight ) )
        
i1 = 0
i2 = 5000
plt.plot( data[i1:i2,1], label='laser piezo voltage' )
plt.plot( d[i1:i2], label='smooth' )
plt.plot( maskLeft[i1:i2], label='left flank boolean' )
plt.plot( maskRight[i1:i2], label='right flank boolean' )
plt.legend()
plt.grid()
plt.show()


In [ ]:
#   3. Plot the mean transmission as a function of binned laser fine-tuning voltages, separate left and right flanks of movement

binDV = 0.5
binEdgesV = np.arange( dMinLevel, dMaxLevel + 0.5 * binDV, binDV )
binCentersV = binEdgesV[0:-2] + 0.5 * binDV
binN = len( binCentersV )
binMeanIntensityV = np.zeros( binN )

for i in range( binN ):
    ind = np.logical_and( d >= binEdgesV[i], d < binEdgesV[i+1] )    
    binMeanIntensityV[i] = np.mean( data2[ind,0] )

binLeftMeanIntensityV = np.zeros( binN )
for i in range( binN ):
    ind = np.logical_and( d >= binEdgesV[i], d < binEdgesV[i+1] )      
    ind = np.logical_and( ind, maskLeft )    
    binLeftMeanIntensityV[i] = np.mean( data2[ind,0] )
     
binRightMeanIntensityV = np.zeros( binN )
for i in range( binN ):
    ind = np.logical_and( d >= binEdgesV[i], d < binEdgesV[i+1] )    
    ind = np.logical_and( ind, maskRight )    
    binRightMeanIntensityV[i] = np.mean( data2[ind,0] )        


plt.plot( binCentersV, binMeanIntensityV, label='all data' )
plt.plot( binCentersV, binLeftMeanIntensityV, label='left flanks' )
plt.plot( binCentersV, binRightMeanIntensityV, label='right flanks' )
plt.xlabel('laser fine-tuning in bins (V)')
plt.ylabel('mean transmission, photodiode  (V)')
plt.grid()
plt.legend()
plt.show()

In [ ]:
# Fit Gauss

# import pylab as plb
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy import asarray as ar,exp


def gaus(x,a,x0,sigma):
    return a*exp(-(x-x0)**2/(2*sigma**2))


x = binCentersV
y = binLeftMeanIntensityV
n = len(x)                          
mean = sum(x*y)/n                   
sigma = sum(y*(x-mean)**2)/n        
popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
binLeftMeanIntensityVFit = gaus(x,*popt)
binLeftMeanIntensityVFitSigma = popt[2]

x = binCentersV
y = binRightMeanIntensityV
n = len(x)                          
mean = sum(x*y)/n                   
sigma = sum(y*(x-mean)**2)/n        
popt,pcov = curve_fit(gaus,x,y,p0=[1,mean,sigma])
binRightMeanIntensityVFit = gaus(x,*popt)
binRightMeanIntensityVFitSigma = popt[2]


fig = plt.figure(figsize=(16, 8))
plt.plot( binCentersV, binLeftMeanIntensityV, label='left flanks' )
plt.plot( binCentersV, binLeftMeanIntensityVFit, label='fit, sigma {} V'.format(binLeftMeanIntensityVFitSigma), linestyle=':' )
plt.plot( binCentersV, binRightMeanIntensityV, label='right flanks' )
plt.plot( binCentersV, binRightMeanIntensityVFit, label='fit, sigma {} V'.format(binRightMeanIntensityVFitSigma), linestyle=':'  )
plt.xlabel('laser fine-tuning in bins (V)')
plt.ylabel('mean transmission, photodiode  (V)')
plt.grid()
plt.legend()
plt.show()




In [ ]:
# From slow piezo tuning, linear approx, we find 50 GHz / 12 V
#
# This would give linewidth of ~ 2.6 V * (50 GHz / 12 V) = 12 GHz.

## Laser modulation

In [ ]:
import glob

dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170912/' 
files = os.listdir( dataDir )[1:]


for file in files:    
    # load data
    filename = os.path.join( dataDir, file, file + '.dat' )        
    data = pandas.read_csv(filename,header=None,skiprows=4,sep='\t').values
    # clean up outlyers 
    ind = np.logical_and( data[:,1] < 3000,  data[:,1] > 1200 )
    data = data[ind,:]   
    # plot
    plotInd1 = 0
    plotInd2 = len( data )
    plt.plot( data[plotInd1:plotInd2,0], data[plotInd1:plotInd2,1] )
    plt.grid()
    plt.xlabel( 'time (s)' )
    plt.ylabel( 'laser frequency (relative, GHz)' )
    plt.title( file )    
    plt.show()
    
    



## Tests if we can fine-tune cavity length to max transmission before slow linewidth measurement

In [ ]:
data_dir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170911/scope' 
file_name = "SCOPE_007.wdf.csv"  
filename = os.path.join(data_dir,file_name)

data, timeResolution = read_scope_xviewer_csv(filename)
time = timeResolution * np.arange(data.shape[0]) 

print '{} traces, time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
    data.shape[1], timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )

In [ ]:
plt.plot( time, data )
plt.show()


freq, power = scipy.signal.welch(data, axis = 0, fs = 1/timeResolution,nperseg = len(data), detrend = 'linear' )
plt.plot( freq[0:300], power[0:300] )

plt.show()

In [ ]:
plt.plot( time[0:5000], data[0:5000] )
plt.show()


plt.plot( time, data )
plt.show()

time1Resolution = 1.0
downSampleFactor = int( time1Resolution/timeResolution )
data1 = data.reshape(-1, downSampleFactor).mean(axis=1)
time1 = np.arange( 0, len( data1) ) * time1Resolution

time2Resolution = 2.0
downSampleFactor = int( time2Resolution/timeResolution )
data2 = data.reshape(-1, downSampleFactor).mean(axis=1)
time2 = np.arange( 0, len( data2) ) * time2Resolution

time3Resolution = 4.0
downSampleFactor = int( time3Resolution/timeResolution )
data3 = data.reshape(-1, downSampleFactor).mean(axis=1)
time3 = np.arange( 0, len( data3) ) * time3Resolution


plt.plot( time1, data1 )
plt.plot( time2, data2 )
plt.plot( time3, data3 )
plt.plot()

print downSampleFactor 
print 2.5e3 / downSampleFactor

